# InsightIQ: Meme Coin ML — End-to-End Demo

In [ ]:

# Setup (run once)
%pip install -q -r requirements.txt
import nltk
nltk.download('vader_lexicon')


## 1) Ingest CoinGecko Prices

In [ ]:

!python src/ingest/fetch_prices.py --coins DOGE,PEPE --days 180 --interval daily --out data/prices.parquet
import pandas as pd
prices = pd.read_parquet('data/prices.parquet')
prices.head()


## 2) Build Sentiment Features

In [ ]:

!python src/ingest/make_sentiment_features.py --in data/sentiment.csv --out data/sentiment_features.parquet
import pandas as pd
sent = pd.read_parquet('data/sentiment_features.parquet')
sent.head()


## 3) Engineer Features & Target

In [ ]:

!python src/features/build_features.py --prices data/prices.parquet --sent data/sentiment_features.parquet --out data/features.parquet
import pandas as pd
feat = pd.read_parquet('data/features.parquet')
feat.head()


## 4) Plot Price & Sentiment

In [ ]:

import pandas as pd, matplotlib.pyplot as plt
df = prices.merge(sent, on='date', how='left')
df = df[df['coin']=='DOGE'].sort_values('date')
plt.figure()
plt.plot(pd.to_datetime(df['date']), df['price_usd'])
plt.title('DOGE Price (USD)'); plt.xlabel('Date'); plt.ylabel('Price'); plt.show()
plt.figure()
plt.plot(pd.to_datetime(df['date']), df['comp_mean'].fillna(method='ffill'))
plt.title('Daily Sentiment (VADER compound)'); plt.xlabel('Date'); plt.ylabel('Compound'); plt.show()


## 5) Train & Register Model (MLflow)

In [ ]:

# Optional: set remote MLflow server
# %env MLFLOW_TRACKING_URI=http://127.0.0.1:5001
!python src/models/train.py --features data/features.parquet --register_name MemeCoinAlpha


## 6) Load the Registered Model and Predict

In [ ]:

import os, mlflow, mlflow.pyfunc, pandas as pd
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI','http://127.0.0.1:5001'))
client = mlflow.tracking.MlflowClient()
vers = client.search_model_versions("name='MemeCoinAlpha'")
version = max(int(v.version) for v in vers) if vers else 1
model = mlflow.pyfunc.load_model(f"models:/MemeCoinAlpha/{version}")

X = pd.read_parquet('data/features.parquet').head(5)[['ret_1d','vol_3d','vol_7d','vol_14d','vol_30d','ret_3d','ret_7d','ret_14d','ret_30d','ma_3d','ma_7d','ma_14d','ma_30d','comp_mean','pos_mean','neg_mean','neu_mean','n_posts']]
model.predict(X)
